<a href="https://colab.research.google.com/github/decoz/mlclass/blob/master/ch3_IMDB_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMDB dataset 

이번에는 텍스트 문서를 학습하는 사례로서 keras.dataset 에서 제공하는 영화 리뷰 데이터, imdb 데이터를 학습시켜보자. 이 데이터는 imdb 의 리뷰들을 부정적인 리뷰와 긍정적인 리뷰로 구분한 데이터로서 이진 분류의 예제로 사용된다. 


<hr style = "height:2px">

## imdb 데이터 학습시키기 


### 데이터 로딩 

데이터를 읽어들이는 과정은 이전 챕터에서 설명한 Minist data set과 거의 유사하다.  

In [0]:
from keras.datasets import imdb 
import keras
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)
print(train_data.shape, test_data.shape)                                                    

In [0]:
print(train_data[0])

imdb 리뷰데이터는 25000 개의 학습데이터와 25000 개의 테스트데이터로 이루어져 있다. 

### 데이터 변환 

로딩된 데이터는 자주 등장하는 1000 단어를 0~999 의 숫자로 변환하여 배열로 기록한 것이다. 예를 들어 

    I : 33
    ..
    love : 47
    ..
    her : 25 
    
    
만일 단어의 인덱스가 위와 같다고 가정하면  I love her 라는 문장은 [ ... 33, 37, 25, ... ] 라는 배열로 표현된다. 각각의 리뷰을 단어에 대응되는 숫자코드로 표현한 것이 imdb dataset 이다. 

텍스트의 특성상 리뷰의 내용과 길이가 다 다르기 때문에 배열의 길이도 각 데이터마다 다르다. 하지만 신경망의 입력은 항상 동일한 크기를 유지해야 하기 때문에 이 데이터는 바로 순차 신경망의 입력으로 사용할 수 없으며 이를 변환해 주어야 한다.  

가장 간단한 방법은 0~9999 번의 단어가 각각 리뷰에 존재하면 0, 존재하지 안으면 1로 표기하는 것이다. for문을 돌려서 데이터를 변환할 수도 있겠지만 이런 경우를 위해 keras 에서는 tokenizer 클래스를 제공한다. 이 클래스의 간단한 사용예를 보자. 


In [0]:
import numpy as np
seq = [[1,2,2,3,4],[1,3,5,7,9]]
tokenizer = keras.preprocessing.text.Tokenizer(num_words = 10) 
mat = tokenizer.sequences_to_matrix(seq, mode='binary')
print(mat)

위의 예제는 총 10개의 단어를 인덱스화 시킨 것으로 각각이 존재하는가는 길이 10의 배열로 표기하게 된다.

로딩된 imdb 의 데이터는 전체 1000개의 인덱스를 제공하므로 각각의 리뷰는 길이 1000 의 배열로 표기할 수 있다. 이제 imdb 데이터를 고정된 길이 1000의 배열로 변환하도록 하겠다.  

In [0]:
import numpy as np 
from keras.preprocessing.text import Tokenizer
tokenizer = keras.preprocessing.text.Tokenizer(num_words = 10000) 
x_train = tokenizer.sequences_to_matrix(np.array(train_data), mode='binary')
x_test = tokenizer.sequences_to_matrix(np.array(test_data), mode='binary')


In [0]:
print(train_labels[:100])

label 의 값은 0~1의 숫자로 표기되며 0은 부정적 1은 긍정적인 리뷰임을 의미한다. 이 역시 [0,1] 과 [1,0] 으로 표현하는 것이 더 좋기 때문에 Minister 데이터와 마찬가지로 keras.utils.to_categorical 함수를 사용하도록하겠다. 

In [0]:
y_train = keras.utils.to_categorical(train_labels, 2)
y_test = keras.utils.to_categorical(test_labels, 2)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

print(y_train)


### 모델 생성  

이제 학습 모델을 생성하자. 입력 레이어의 input_shape 는  (1000,) 으로 되어야 한다. 히든 레이어는 1000 개의 입력을 적절히 수용할 수 있도록 128개의 노드를 지닌 하나의 층으로 설정하도록 하였다. 입력과 히든은 수치적인 민감도가 낮기 때문에 activation 함수는 디폴드값인 'relu' 로 설정하였고 출력은 분류모델의 국민 activation 함수라고 할수 있는 'softmax' 로 설정하였다. 


In [0]:
import keras 
from keras import models
from keras import layers 
keras.backend.clear_session()

model = models.Sequential() 
model.add( layers.Dense(128, input_shape = (10000,)))
model.add( layers.Dense(128))
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])


### 학습 

이제 준비가 모두 되었으니 학습을 시작해 보도록 하겠다. 학습 셋의 크기가 25000으로 매우 큰 편이기때문에 이번엔 배치 사이즈를 좀 크게 잡아 1000 정도로 세팅하였다. 

In [0]:
history = model.fit(x_train, y_train, epochs = 10, batch_size = 100)

### 데이터 검증 

Mnist 데이터와 마찬가지로  model.evaluat() 를 이용해 검증을 하면 다음과 같다. 


In [0]:
test_loss, test_acc =  model.evaluate(x_test, y_test)
print(test_loss, test_acc)

<font color = 'red'>
    
###  연습문제 3-1 : 정확도 변화 보기 
    
복습겸 Mnist 데이터때의 내용을 참조하여 학습 정확도 변화를 그래프로 출력해보자. 

In [0]:
import matplotlib.pyplot as plt 
# 여기에 위의 그래프를 시각화시켜보세요



<hr style = "height:2px">

## 학습과 검증을 동시 진행 

실용적인 학습모델을 구축하기 위해서는 학습의 수렴여부만큼 과적합의 여부도 매우 중요하다. 특히 데이터가 충분하지 안은 경우 과적합은 매우 빈번히 발생하기 때문에 현재 학습된 코드가 과적합에 빠지는지 여부를 모니터링 할 필요가 있다. 그러나 이를 위해 매번 epoch 를 멈추고  evaluate 를 수행하는 것은 비효율적이기 때문에 model.fit 에서는 validation_data 라는 옵션을 이용해 매 epoch 마다 현재 모델의 테스트 데이터에 대한 평가를 수행할 수 있다.  

사실 imdb 데이터의 경우 테스트 데이터가 학습 데이터의 숫자와 동일하게 매우 크므로  매번 이를 검사하는 것은 약간 비효율적일 수도 있으나 간단한 모델로 이를 측정해보도록 하겠다. 


In [0]:
history = model.fit(x_train, y_train, epochs = 40, batch_size = 1000,  validation_data = (x_test, y_test), verbose = 1).history
import matplotlib.pyplot as plt


아까와는 다르게 loss, acc 외에도 val_loss , val_acc 가 출력되는 것을 볼 수 있을 것이다. 이것이 train 데이터가 아닌 test 데이터를 이용하여 평가한 정확도이며 실제 모델의 사용시 더 중요한 척도가 된다. 

대부분은 이러한 val_loss 와 val_acc 가 loss, acc 에 비해 빨리 수렴하거나 학습효율이 떨어지는걸 체감할 수 있을 것이다. 이를 도식화시켜서 보도록 하자. 

In [0]:
plt.plot(history['loss'], label = 'loss')            
plt.plot(history['val_loss'], label = 'val_loss')
plt.legend()
plt.show()

plt.plot(history['acc'], label = 'acc')            
plt.plot(history['val_acc'], label = 'val_acc')
plt.legend()
plt.show()

print("last acc / val_acc : ", history['acc'][-1], "/",  history['val_acc'][-1])

<font color = 'red'>
    
###  연습문제 3-2 : 중간 레이어 변경
    
중간 레이어를 늘리거나 줄이거나 하며 이것이 학습성능 및 과적합의 여부에 영향을 주는지 한번 테스트해보자. 

In [0]:
# 위의 모델 생성 코드를 여기로 복사해서 수정한 뒤에 성능 평가를 해보세요 


<font color = 'red'>
    
###  연습문제 3-3 :  단어의 크기 변경후 테스트
    
위의 imdb 예제는 단어의 범위를 0~999 로 제한하여 입력열이 1000개로 이뤄졌다. 이를 10000 개로 늘여서 한번 성능을 테스트해보자. 

In [0]:
from keras.datasets import imdb 
import keras
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)
# 그 후의 코드는 직접 카피해서 수정해보세요 






<hr style = "height:2px">

## 실제 imdb 리뷰를 예측하기 


### 데이터의 복원

imdb 리뷰라고 하지만 우리가 읽어들인 것들은 단순한 숫자들의 나열에 지나지 안으므로 사실 실감이 잘 나지 안는다. 각각의 데이터가 실제로 어떤 리뷰였는지 복원 할 수 있을까? 이런 의문에 답하기 위해 imdb 데이터셋은 get_word_index라는 함수를 지원한다. 


In [0]:
wi = imdb.get_word_index() 
print(wi)
print(len(wi))

get_word_index 는 각각의 단어와 매칭되는 숫자 를 키:밸류로 조합된 dict 로  리턴한다.  참고로 위의 결과를 보면 실제 imdb 데이터에서 제공하는 전체 단어테이블의 크기는 88584 라는 것을 알 수 있다. 
이 인덱스 테이블을 통해 우리는 각각의 데이터를 리뷰 단어들로 부분적으로나마 복원할 수 있다.  다음은 train_data[0] 의 복원과정과 그 결과이다. 

In [0]:
print(train_data[2])

wi = imdb.get_word_index() 
rwi = dict( [(v,k) for (k,v) in wi.items()] )
review = ' '.join([rwi.get(i-3, '?') for i in train_data[2]])

print(review)



중간 중간 ? 로 표기되는 것은 단어테이블에 존재하지 안는 단어를 의미한다. 수많은 고유명사까지 포함하면 사실 9만개 내외의 숫자로 모든 실제 리뷰의 단어들을 표기하는 것은 불가능하기 때문에 등장빈도가 낮은 단어는 누락되었음을 의미한다.  데이터중에 0~2의 값은 테이블에 존재하지 안는 단어들이다.  또한 여러 특수기호, 마침표, 쉼표등도 삭제되므로 사실 100프로 복원은 어렵다. 


### 실제 리뷰를 예측용 데이터로 변환

이번에는 직접 간단한 리뷰를 작성해 train_set 과 같은 코드배열로 변환해보도록 하겠다. 


In [0]:
my_review = """
    The plot and themes of this movie are very complex and work very well. Also technical aspects of this film are top notch. This is damaged however by some terrible styleistic choices, like having Brad Pitt narrating over every action scene and slow piano music constantly playing, making the film feel really slow and boring.
    """

words = [ wi.get(w,2) for w in my_review.split() ]
print(words)

짧지만 이것은 train_data 의 각 리뷰의 포맷과 같은 형태이다. 이제 이 배열을 위의 학습셋 변환과 동일하게 1000짜리 배열로 변환한다. 

In [0]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words = 10000) 
my_x  = tokenizer.sequences_to_matrix([words], mode='binary')
print(my_x)

이제 이것을 predict 를 사용하여 결과값을 뽑아보도록 하겠다. 

In [0]:
r = model.predict(my_x)
print(r)
print("review is ", np.argmax(r))

<font color = 'red'>
    
###  연습문제 3-3 : reuter dataset 학습
    
imdb 와 거의 유사한 문제로 로이터 연합의 기사에 대한 dataset이 제공된다. 이 데이터를 학습하도록 해보자. 이 데이터는 거의 imdb datasets와 유사하지만 그 결과값이 46가지 토픽을 예측하도록 되어있다. 또한 imdb와 달리 테스트셋의 크기가 트레인셋보다 작은 편이다. 하지만 나머지는 거의 유사하므로 imdb의 코드를 참조하면 쉽게 학습을 수행할 수 있을 것이다. 





<hr style = "height:2px">

## 학습 제어하기 

model.fit 은 자동으로 epoch들을 진행하며 학습을 수행한다. 하지만 만일 학습이 상당히 오랜 시간을 필요로 한다고 할때 만일 중간에 학습이 발산하거나 과적합되어 버리면 많은 시간을 낭비하게 된다.  또한 꺼꾸로 빠른 시점에 만족스러운 성능이 나온다면 나머지 학습의 시간이 비효율적으로 사용될 수도 있다.  

이런 여러 상황을 대처하기 위해 개발자가 직접 학습 중간중간을 제어할 수 있는 코드를 넣을 수 있다. 자동으로 작동되는 체계에 우리가 원하는 코드를 삽입하기 위해서는 개발자가 작성한 함수를 인자값으로 주는 일종의 콜백이 필요하다. 케라스 역시 이러한 콜백함수(정확히는 콜백클래스)를 넘겨줄 수 있도록 되어있다. 

    model.fit(..., callbacks = [콜백 클래스들] ) 
    
이 콜백클래스는 keras.callbacks.Callback 을 상속받아 작성하게 되는데 간단한 예는 다음과 같다. 

In [0]:
class check(keras.callbacks.Callback):
    def init(self):
        print("start epoch")         
    def on_epoch_end(self, batch, logs={}):
        print(batch,"- loss:", logs.get('loss'), "acc:", logs.get('acc') )


이제 이를 이용해 위의 학습을 콜백함수의 출력으로 대처해 작동시켜보겠다. 

In [0]:
history = model.fit(x_train, y_train, epochs = 10, batch_size = 1000,  
                    callbacks = [check()],
                    validation_data = (x_test, y_test), verbose = 0).history


만일 특정 조건이 만족되었을때 학습을 종료하고 싶다면 다음과 같은 코드를 사용하면 된다. 
       
    self.model.stop_training = True
            
이제 imdb 의 정확도가 0.87에 다다르면 끝내는 코드를 작성하면 다음과 같다. 

In [0]:
class check(keras.callbacks.Callback):
    def init(self):
        print("start epoch")         
    def on_epoch_end(self, batch, logs={}):
        print(batch,"- loss:", logs.get('loss'), "acc:", logs.get('acc') )
        if logs.get('acc') > 0.87 :
            self.model.stop_training = True

history = model.fit(x_train, y_train, epochs = 10, batch_size = 1000,  
            callbacks = [check()],
            validation_data = (x_test, y_test), verbose = 0).history            